In [38]:
!pip install gurobipy

In [208]:
import pandas as pd
from gurobipy import *


In [286]:
# Reading the data from the excel sheets
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name="PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name="PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name="Truck")

# Convert relevant columns to datetime and normalize shift times
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])



In [288]:
# Initialize the model
model = Model("DHL_Optimization")

# Decision Variables
source_list = list(trucking_df['Origin_ID'].unique()[:3])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:3])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list = [
    x for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]

valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(300)

# Decision Variables
X = {}
Z = {}
T = {}
ArrivalDay = {}
ArrivalTime = {}

for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]:
    X[(i, j, k, l)] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}_{l}")
    

for l in trucks:
    Z[l] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}")
    # T[l] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"T_{l}")
    # ArrivalDay[l] = model.addVar(lb = 0,ub = 3, vtype=GRB.INTEGER, name=f"ArrivalDay_{l}")
    # ArrivalTime[l] = model.addVar(lb=0, ub=24, vtype=GRB.CONTINUOUS, name=f"ArrivalTime_{i}_{j}_{k}_{l}")


In [294]:
len(consignment_list)

153

In [289]:
# Objective function: Minimize the total arrival time
model.setObjective(quicksum(X[i, j, k, l] for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]), GRB.MINIMIZE)

In [290]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations) <= 2 * Z[l])

In [255]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model.addConstr(T[l] >= release_time * X[(i, j, k, l)])

In [256]:
# 3. Truck must arrive at the destination within the operational hours
for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    for l in trucks:
        model.addConstr(ArrivalTime[(l)] == (T[l] + travel_time * X[(i, j, k, l)] + 24 * ArrivalDay[l]) - 24 * model.addVar(lb = 0, vtype=GRB.INTEGER, name=f"multiplier_{i}_{j}_{k}_{l}"))
        model.addConstr(ArrivalTime[(l)] >= start_shift)
        model.addConstr(ArrivalTime[(l)] <= end_shift)

In [291]:
# 4. Each consignment must be assigned to exactly one truck
for (i, j, k) in valid_combinations:
    model.addConstr(quicksum(X[(i, j, k, l)] for l in trucks) == 1)

In [276]:
#6. Flow conservation: If a truck leaves a source, it must go to one destination
for l in trucks:
    for i in source_list:
        model.addConstr(quicksum(X[(i, j, k, l)] for j in destination_list if j != i for k in consignment_list if (i, j, k) in valid_combinations) >= Z[l])

In [223]:
# 6. Sorting Capacity: Each PZE should have enough capacity to accommodate all the incoming trucks
for j in destination_list:
    working_hours = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0] - destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    model.addConstr(quicksum(X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0]
                      for i in source_list if j != i for k in consignment_list for l in trucks if (i, j, k) in valid_combinations ) <= working_hours * destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0])

In [201]:
# model.setParam('TimeLimit', 100)

In [292]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 453 rows, 46200 columns and 92100 nonzeros
Model fingerprint: 0xb03d5cd7
Variable types: 0 continuous, 46200 integer (46200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 153.0000000
Presolve removed 0 rows and 300 columns
Presolve time: 0.10s
Presolved: 453 rows, 45900 columns, 91800 nonzeros
Variable types: 0 continuous, 45900 integer (45900 binary)

Explored 0 nodes (0 simplex iterations) in 0.14 seconds (0.11 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 153 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.530000000000e+

In [267]:
print('Optimization Status:', model.Status)


Optimization Status: 2


In [283]:
# Print decision variables
print("\nDecision Variables:")
for v in model.getVars():
    try:
        print(f"{v.VarName} = {v.X}")
    except AttributeError:
        print(f"{v.VarName} = No value assigned (variable may not be optimized)")


Decision Variables:
X_01.1.1.PZ_04.1.1.PZ_7791768_0 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_1 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_2 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_3 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_4 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_5 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_6 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_7 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_8 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_9 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_10 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_11 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_12 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_13 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_14 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_15 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_16 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_17 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_18 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_19 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_20 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_21 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_22 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768_23 = -0.0
X_01.1.1.PZ_04.1.1.PZ_7791768

In [285]:
# Accessing ArrivalDay values after optimization
if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SUBOPTIMAL]:
    # Extract the data into a DataFrame
    data = []
    for (i, j, k, l) in X.keys():
        if X[i, j, k, l].X > 0:
            travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600
            arrival_time = T[l].X + travel_time
            data.append({
                'i': i,
                'j': j,
                'k': k,
                'l': l,
                'Z':Z[l].X,
                # 'T(l)': T[l].X,
                # 'ArrivalTime': ArrivalTime[l].X,
                # 'ArrivalDay': ArrivalDay[l].X
            })

KeyError: 232

In [261]:
df = pd.DataFrame(data)

In [250]:
df.head()

,i,j,k,l,Z,T(l),ArrivalTime,ArrivalDay
0,01.1.1.PZ,04.1.1.PZ,7791768,0,1.0,22.0,22.0,-0.0
1,01.1.1.PZ,04.1.1.PZ,7791768,1,1.0,22.0,22.0,0.0
2,01.1.1.PZ,04.1.1.PZ,7791768,2,1.0,22.0,22.0,0.0
3,01.1.1.PZ,04.1.1.PZ,7791768,3,1.0,22.0,22.0,0.0
4,01.1.1.PZ,04.1.1.PZ,7791768,4,1.0,22.0,22.0,0.0


In [174]:
df.shape

(45900, 7)